In [35]:
import numpy as np
from collections import Counter
import math
def _entropy( class_probabilities: list) -> float:
    return sum([-p * np.log2(p) for p in class_probabilities if p>0])
    
def _class_probabilities( labels: list) -> list:
    total_count = len(labels)
    return [label_count / total_count for label_count in Counter(labels).values()]

def _data_entropy(labels: list) -> float:
    return _entropy(_class_probabilities(labels))
    

In [36]:
_data_entropy([0.1,0.1,0.5,0.2,0.3])

1.9219280948873623

In [37]:
 def _partition_entropy( subsets: list) -> float:
        """subsets = list of label lists (EX: [[1,0,0], [1,1,1])"""
        total_count = sum([len(subset) for subset in subsets])
        return sum([_data_entropy(subset) * (len(subset) / total_count) for subset in subsets])
    

In [38]:
_partition_entropy([[1,0,0], [1,1,1]])


0.4591479170272448

In [39]:
from collections import defaultdict

In [40]:
def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy""" 
    return sum(-p * math.log(p, 2)
        for p in class_probabilities if p)
def class_probabilities(labels): 
    total_count = len(labels) 
    return [count / total_count
        for count in Counter(labels).values()]
def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data] 
    probabilities = class_probabilities(labels) 
    return entropy(probabilities)

In [41]:
def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets subsets is a list of lists of labeled data"""
    total_count = sum(len(subset) for subset in subsets)
    return sum( data_entropy(subset) * len(subset) / total_count for subset in subsets )

In [42]:

def partition_by(inputs, attribute):
    """each input is a pair (attribute_dict, label). returns a dict : attribute_value -> inputs""" 
    groups = defaultdict(list)
    for input in inputs:
        key = input[0][attribute]
        groups[key].append(input) 
    return groups

In [46]:
 inputs = [
        ({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'},False),
        ({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'},False),
        ({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'},True),
        ({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'},True),
        ({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'},True),
        ({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'},False),
        ({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'},True),
        ({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'},  False),
        ({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'},       True),
        ({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'},  True),
        ({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
        ({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'},     True),
        ({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'},       True),
        ({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False)
]

In [56]:
def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition""" 
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())


In [57]:
for key in ['level','lang','tweets','phd']: 
    print (key, partition_by(inputs, key))

level defaultdict(<class 'list'>, {'Senior': [({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'no'}, False), ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'yes'}, False), ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, False), ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True), ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'yes'}, True)], 'Mid': [({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True), ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, True), ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, True), ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd': 'no'}, True)], 'Junior': [({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True), ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True), ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, False), ({'level': 'Junior', 'lang': 'Python', 'twee

In [60]:
from functools import partial
def build_tree_id3(inputs, split_candidates=None):
        if split_candidates is None:
            split_candidates = inputs[0][0].keys()
        num_inputs = len(inputs)
        num_trues = len([label for item, label in inputs if label]) 
        num_falses = num_inputs - num_trues
        if num_trues == 0: return False 
        if num_falses == 0: return True
        if not split_candidates:
            return num_trues >= num_falses # return the majority leaf
                # otherwise, split on the best attribute
        best_attribute = min(split_candidates,
                                     key=partial(partition_entropy_by, inputs))
        partitions = partition_by(inputs, best_attribute) 
        new_candidates = [a for a in split_candidates if a != best_attribute]
        subtrees = { attribute_value : build_tree_id3(subset, new_candidates) for attribute_value, subset in partitions.items() }
        subtrees[None] = num_trues > num_falses # default case return (best_attribute, subtrees)
        return (best_attribute, subtrees)

In [61]:
tree = build_tree_id3(inputs)


In [62]:
tree

('level',
 {'Senior': ('tweets', {'no': False, 'yes': True, None: False}),
  'Mid': True,
  'Junior': ('phd', {'no': True, 'yes': False, None: True}),
  None: True})

In [69]:
def classify(tree, input):
    if tree in [True, False]: 
        return tree
    attribute, subtree_dict = tree
    subtree_key = input.get(attribute) 
    print(subtree_key)
    if subtree_key not in subtree_dict:
        subtree_key = None
    subtree = subtree_dict[subtree_key]
    print(subtree)
    return classify(subtree, input)

In [70]:
classify(tree, { "level" : "Junior",
                 "lang" : "Java",
                 "tweets" : "yes",
                 "phd" : "no"} )

Junior
('phd', {'no': True, 'yes': False, None: True})
no
True


True